In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/BraTS2021_00495.tar
/kaggle/input/BraTS2021_Training_Data.tar
/kaggle/input/BraTS2021_00621.tar


In [3]:
import tensorflow as tf
# detect and init the TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

    # instantiate a distribution strategy
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.TPUStrategy(tpu)
except:
    pass
%pip install glob
%pip install nibabel

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:

import tarfile
file = tarfile.open('/kaggle/input/BraTS2021_Training_Data.tar')

file.extractall('./BraTS2021_Training_Data')
file.close()


In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all messages are logged (default behavior)
                                          # 1 = INFO messages are not printed
                                          # 2 = INFO and WARNING messages are not printed
                                          # 3 = INFO, WARNING, and ERROR messages are not printed
import tensorflow as tf
tf.config.run_functions_eagerly(False)
tf.get_logger().setLevel('ERROR')


In [ ]:

import tensorflow.keras as kr
import nibabel as nib
import numpy as np
import os
from glob import glob
import matplotlib.pylab as plt

folder_path = [os.path.join(r"BraTS2021_Training_Data", i) for i in os.listdir(r"BraTS2021_Training_Data")]
imgs = []


for i in folder_path:
    imgs.append(glob(i+"/*"))
print("Done")
folder_path = folder_path[1:]
imgs = imgs[1:]
print(imgs[0])



#input directory and output directory
outputpath = []
inputpath = []
for j in imgs:
    for i in j:
        if "seg.nii.gz" in i.split("_"):
            outputpath.append(i)
for j in imgs:
    temppath = []
    for i in j:
        if "seg.nii.gz" not in i.split("_"):
            temppath.append(i)
    inputpath.append(temppath)
    
x_train_path = inputpath[:1126].copy()
y_train_path = outputpath[:1126].copy()
x_eval_path = inputpath[1126:1189].copy()
y_eval_path = outputpath[1126:1189].copy()
x_test_path = inputpath[1189:].copy()
y_test_path = outputpath[1189:].copy()  

parition = {"train": x_train_path, "validation": x_eval_path}
label = {"train": y_train_path, "validation": y_eval_path}
#---------------------------------------------------------------------------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------OLD-------------------------------------------------------------------------------
# def prepoutput():
#     index = 0
#     outputimg = np.array((nib.load(outputpath[0]).get_fdata().astype(np.float32)[:, :, 75]))
#     outputimg = np.expand_dims(outputimg, axis=2)
#     outputimg = tf.image.resize(outputimg, (256, 256))
#     for i in range(1,len(outputpath)):
#         index+=1
#         print(index)
#         x = nib.load(outputpath[i]).get_fdata().astype(np.float32)[:, :, 75]
#         x = np.expand_dims(x, axis=2)
#         x = tf.image.resize(x, (256, 256))
#         outputimg = np.dstack((outputimg,x),)
#     return outputimg
# outputimg = prepoutput()
# outputimg = outputimg.transpose((2, 0, 1))
# np.save('output_dataset.npy', outputimg)

#


# inputimg = nib.load(inputpath[0][0]).get_fdata().astype(np.float32)[:, :, 75]
# for i in range(1,len(inputpath[0])):
#     inputimg = np.dstack((inputimg,nib.load(inputpath[0][i]).get_fdata().astype(np.float32)[:, :, 75]))
# inputimg = np.expand_dims(inputimg, axis=0)

# for i in range(1,len(inputpath)):
#     print(i)
    
#     temp = nib.load(inputpath[i][0]).get_fdata().astype(np.float32)[:, :, 75]
#     print("\t1")
#     print(f"\t{inputpath[i][0]}")
#     for channels in range(1,len(inputpath[i])):
#         print(f"\t{channels+1}")
#         print(f"\t{inputpath[i][channels]}")
#         temp = np.dstack((temp,nib.load(inputpath[i][channels]).get_fdata().astype(np.float32)[:, :, 75]))
        
#     temp = np.expand_dims(temp, axis=0)
#     inputimg = np.concatenate((inputimg,temp), axis=0)
#     print(inputimg.shape)
#---------------------------------------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------------------------------------------------------

import numpy as np
import keras
import random
import time
import os
import nibabel as nib

import numpy as K



import tensorflow.keras as keras
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
#set up var
x_VolSlice_start = 10
x_VolSlice_end = 150

y_VolSlice_start = 10
y_VolSlice_end = 150


#-----------------------------------------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------model making--------------------------------------------------------------------------------

#note:
# conv3d need a 5d tensor (height,width,depth,channels)
tf.experimental.numpy.experimental_enable_numpy_behavior()

import tensorflow as tf
from tensorflow.keras import layers, Model
# dice loss as defined above for 4 classes
def dice_coef(y_true, y_pred, smooth=1.0):
    class_num = 4
    for i in range(class_num):
        try:
            y_true_f = np.array((y_true[:,:,:,i])).flatten()
            y_pred_f = np.array((y_pred[:,:,:,i])).flatten()
        except:
            y_true_f = (y_true[:,:,:,i]).reshape(-1)
            y_pred_f = (y_pred[:,:,:,i]).reshape(-1)
            
            
        intersection = K.sum(y_true_f * y_pred_f)
        loss = ((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))
   #     K.print_tensor(loss, message='loss value for class {} : '.format(SEGMENT_CLASSES[i]))
        if i == 0:
            total_loss = loss
        else:
            total_loss = total_loss + loss
            
    total_loss = total_loss / class_num
#    K.print_tensor(total_loss, message=' total dice coef: ')
    return total_loss


 
# These functions are used for evaluating the performance of a segmentation model on three different classes
# in medical imaging (presumably related to brain tumor segmentation).
# Input Parameters:
# y_true: The ground truth segmentation mask for the edema class.
# y_pred: The predicted segmentation mask for the edema class.
# epsilon: A small constant to avoid division by zero.

def dice_coef_necrotic(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true[:,:,:,1] * y_pred[:,:,:,1]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,1])) + K.sum(K.square(y_pred[:,:,:,1])) + epsilon)

def dice_coef_edema(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true[:,:,:,2] * y_pred[:,:,:,2]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,2])) + K.sum(K.square(y_pred[:,:,:,2])) + epsilon)

def dice_coef_enhancing(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true[:,:,:,3] * y_pred[:,:,:,3]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,3])) + K.sum(K.square(y_pred[:,:,:,3])) + epsilon)



# Computing Precision 
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    
# Computing Sensitivity      
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())


# Computing Specificity
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())


from tensorflow.keras import layers

def conv_block_3d(input_tensor, num_filters):
    x = layers.Conv3D(num_filters, 3, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv3D(num_filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

def unet_3d(input_shape=(156, 156, 140, 4), num_classes=4):
    inputs = layers.Input(input_shape, dtype='float16')
    
    # Encoder (downsampling)
    conv1 = conv_block_3d(inputs, 32)
    pool1 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv1)
    
    conv2 = conv_block_3d(pool1, 64)
    pool2 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv2)
    
    conv3 = conv_block_3d(pool2, 128)
    pool3 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv3)
    
    conv4 = conv_block_3d(pool3, 256)
    pool4 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv4)
    
    # Bridge
    conv5 = conv_block_3d(pool4, 512)
    
    # Decoder (upsampling)
    up6 = layers.Conv3DTranspose(256, 2, strides=(2, 2, 2), padding='same')(conv5)
    up6 = layers.Lambda(lambda x: tf.pad(x, [[0, 0], [0, conv4.shape[1] - x.shape[1]], [0, conv4.shape[2] - x.shape[2]], [0, conv4.shape[3] - x.shape[3]], [0, 0]]))(up6)
    concat6 = layers.Concatenate()([up6, conv4])
    conv6 = conv_block_3d(concat6, 256)
    
    up7 = layers.Conv3DTranspose(128, 2, strides=(2, 2, 2), padding='same')(conv6)
    up7 = layers.Lambda(lambda x: tf.pad(x, [[0, 0], [0, conv3.shape[1] - x.shape[1]], [0, conv3.shape[2] - x.shape[2]], [0, conv3.shape[3] - x.shape[3]], [0, 0]]))(up7)
    concat7 = layers.Concatenate()([up7, conv3])
    conv7 = conv_block_3d(concat7, 128)
    
    up8 = layers.Conv3DTranspose(64, 2, strides=(2, 2, 2), padding='same')(conv7)
    up8 = layers.Lambda(lambda x: tf.pad(x, [[0, 0], [0, conv2.shape[1] - x.shape[1]], [0, conv2.shape[2] - x.shape[2]], [0, conv2.shape[3] - x.shape[3]], [0, 0]]))(up8)
    concat8 = layers.Concatenate()([up8, conv2])
    conv8 = conv_block_3d(concat8, 64)
    
    up9 = layers.Conv3DTranspose(32, 2, strides=(2, 2, 2), padding='same')(conv8)
    up9 = layers.Lambda(lambda x: tf.pad(x, [[0, 0], [0, conv1.shape[1] - x.shape[1]], [0, conv1.shape[2] - x.shape[2]], [0, conv1.shape[3] - x.shape[3]], [0, 0]]))(up9)
    concat9 = layers.Concatenate()([up9, conv1])
    conv9 = conv_block_3d(concat9, 32)
    
    outputs = layers.Conv3D(num_classes, 1, activation='softmax')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

import tensorflow as tf
tf.keras.backend.clear_session()

print("start")

try:
    with tpu_strategy.scope():
        model = unet_3d(input_shape=(156, 156, 140, 4), num_classes=4)
        model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4), dice_coef, precision, sensitivity, specificity, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing] )
    tf.debugging.set_log_device_placement(True)
except:
    model = unet_3d(input_shape=(156, 156, 140, 4), num_classes=4)
    model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4), dice_coef, precision, sensitivity, specificity, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing] )
    tf.debugging.set_log_device_placement(True)
    
model.summary()




#datagen
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=5, dim=(156,156,140), n_channels=4,
                 n_classes=10, shuffle=True, real_batchsize_custom=2, frames_chunk=18,
                 y_VolSlice_start=0, y_VolSlice_end=None):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.real_batchsize_custom = real_batchsize_custom
        self.frames_chunk = frames_chunk
        self.y_VolSlice_start = y_VolSlice_start
        self.y_VolSlice_end = y_VolSlice_end if y_VolSlice_end is not None else dim[2]

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        sliced_dim = (self.dim[0], self.dim[1], self.y_VolSlice_end - self.y_VolSlice_start)
        X = np.empty((self.real_batchsize_custom, *sliced_dim, 4))
        y = np.empty((self.real_batchsize_custom,*sliced_dim, 4), dtype=float)

        initial_point = 0
        final_point = self.frames_chunk
        for x in range(self.real_batchsize_custom):    
            indexes_orig = indexes[initial_point:final_point]
            list_IDs_temp = [self.list_IDs[k] for k in indexes_orig]
            X_in, y_in = self.__data_generation(list_IDs_temp)
            X[x] = X_in
            y[x] = y_in
            initial_point = final_point
            final_point = final_point + self.frames_chunk   
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        sliced_dim = (self.dim[0], self.dim[1], 140)
        X = np.empty((1, *sliced_dim, 4))
        y = np.zeros((1,*sliced_dim,4))
        
        random.seed(time.time())

        for i, file_paths in enumerate(list_IDs_temp):
            image = self.load_nifti(file_paths)
                
            X[0] = image
            
            
            # Set the label
            seg = nib.load(self.labels[i]).get_fdata().astype(np.float16)[:,:,self.y_VolSlice_start:self.y_VolSlice_end]
            seg[seg==4] = 3

            re= np.expand_dims(seg, axis=0)

            re = tf.image.resize(re,(156,156))
            re = re[0]
            seg = tf.cast(re, tf.int32)
            print(i)
            mask = tf.one_hot(seg, 4);
            
            
            y[0] = mask
            
        X = X.astype(np.float16)
        y = y.astype(np.float16)
        
        return X[0]/np.max(X[0]), y[0]

    def load_nifti(self, file_paths):
        i = 0
        temp = nib.load(inputpath[i][0]).get_fdata().astype(np.float16)[:,:,self.y_VolSlice_start:self.y_VolSlice_end]
        re= np.expand_dims(temp, axis=0)
        re = tf.image.resize(re,(156,156))
        temp = re[0]
        
        temp = np.expand_dims(temp, axis=3)
        for channels in range(1,len(file_paths)):
            tempo = nib.load(file_paths[channels]).get_fdata().astype(np.float16)[:,:,self.y_VolSlice_start:self.y_VolSlice_end]
            re= np.expand_dims(tempo, axis=0)
            re = tf.image.resize(re,(156,156))
            tempo = re[0]
            tempo = np.expand_dims(tempo, axis=3)
            temp = np.concatenate((temp,tempo),axis=3)
        
        return temp
training_generator = DataGenerator(parition["train"],label["train"])
valid_generator = DataGenerator(parition["validation"],label["validation"])

#model fitting


from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

filepath="3D-UNet-2018-weights-improvement-{epoch:02d}-{val_accuracy:.3f}.keras" 

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

csv_logger = CSVLogger('training_2021_2D_UNet.log')

history =  model.fit(training_generator,
                    epochs=12,
                    steps_per_epoch=len(parition["train"]),
                    callbacks= [checkpoint, csv_logger, early_stop],
                    validation_data = valid_generator
                    )

with open('history.txt', 'w') as f:
    f.write(history)
try:
    model.save("final_model.h5")
except:
    model.save("final_model.h5")

Done
['BraTS2021_Training_Data/BraTS2021_00152/BraTS2021_00152_seg.nii.gz', 'BraTS2021_Training_Data/BraTS2021_00152/BraTS2021_00152_flair.nii.gz', 'BraTS2021_Training_Data/BraTS2021_00152/BraTS2021_00152_t1ce.nii.gz', 'BraTS2021_Training_Data/BraTS2021_00152/BraTS2021_00152_t1.nii.gz', 'BraTS2021_Training_Data/BraTS2021_00152/BraTS2021_00152_t2.nii.gz']
start


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 156, 156,  │          0 │ -                 │
│ (InputLayer)        │ 140, 4)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d (Conv3D)     │ (None, 156, 156,  │      3,488 │ input_layer[0][0] │
│                     │ 140, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 156, 156,  │        128 │ conv3d[0][0]      │
│ (BatchNormalizatio… │ 140, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 156, 156,  │          0 │ batch_normalizat… │
│ (Activation)        │ 140, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_1 (Conv3D)   │ (None, 156, 156,  │     27,680 │ activation[0][0]  │
│                     │ 140, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 156, 156,  │        128 │ conv3d_1[0][0]    │
│ (BatchNormalizatio… │ 140, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 156, 156,  │          0 │ batch_normalizat… │
│ (Activation)        │ 140, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d       │ (None, 78, 78,    │          0 │ activation_1[0][… │
│ (MaxPooling3D)      │ 70, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_2 (Conv3D)   │ (None, 78, 78,    │     55,360 │ max_pooling3d[0]… │
│                     │ 70, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 78, 78,    │        256 │ conv3d_2[0][0]    │
│ (BatchNormalizatio… │ 70, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 78, 78,    │          0 │ batch_normalizat… │
│ (Activation)        │ 70, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_3 (Conv3D)   │ (None, 78, 78,    │    110,656 │ activation_2[0][… │
│                     │ 70, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 78, 78,    │        256 │ conv3d_3[0][0]    │
│ (BatchNormalizatio… │ 70, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 78, 78,    │          0 │ batch_normalizat… │
│ (Activation)        │ 70, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_1     │ (None, 39, 39,    │          0 │ activation_3[0][… │
│ (MaxPooling3D)      │ 35, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_4 (Conv3D)   │ (None, 39, 39,    │    221,312 │ max_pooling3d_1[… │
│                     │ 35, 128)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 39, 39,    │        512 │ conv3d_4[0][0]  

 Total params: 22,589,796 (86.17 MB)

 Trainable params: 22,583,908 (86.15 MB)

 Non-trainable params: 5,888 (23.00 KB)

0
1
2
3


model

In [ ]:
model